In [1]:
# Copyright (c) OpenMMLab. All rights reserved.
import argparse

import cv2
import torch
import math


import mmdet
import mmengine
import importlib
importlib.reload(mmdet)
importlib.reload(mmengine)

from mmdet.apis import inference_detector, init_detector
import os
from random import choice
from mmdet.utils.misc import get_test_pipeline_cfg
from mmcv.transforms import Compose
import mmdet.models.backbones

/opt/conda/envs/openmmlab/lib/python3.8/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [837]:
importlib.reload(mmdet.models.backbones.grad_wrapper)

default_config = "configs/yolox/yolo_grad.py"
default_checkpoint = "work_dirs/yolo_poison_rate_20/epoch_200.pth"

# layer_name = "backbone/stage4/2/blocks/0/conv2" # Zadnji u backbone
# layer_name = "backbone/stem/conv"
# layer_name="neck/reduce_layers/0"
layer_name="bbox_head/multi_level_conv_cls/1"
# layer_name="bbox_head/multi_level_conv_obj/0"

cfg_options = dict(
    model=dict(
        layer_name=layer_name,
        bbox_head=dict(num_classes=1)
        ))

model = init_detector(
    default_config,
    default_checkpoint,
    device="cuda",
    cfg_options=cfg_options)
model.eval();

CONFIG LOAD
EVALTYPE clean
DATAROOT data/coco_poisoned_20_mixcolored/
TRAIN STEP True {'type': 'GradWrapper', 'data_preprocessor': {'type': 'DetDataPreprocessor', 'pad_size_divisor': 32, 'batch_augments': []}, 'backbone': {'type': 'CSPDarknet', 'deepen_factor': 0.33, 'widen_factor': 0.5, 'out_indices': (2, 3, 4), 'use_depthwise': False, 'spp_kernal_sizes': (5, 9, 13), 'norm_cfg': {'type': 'BN', 'momentum': 0.03, 'eps': 0.001}, 'act_cfg': {'type': 'Swish'}}, 'neck': {'type': 'YOLOXPAFPN', 'in_channels': [128, 256, 512], 'out_channels': 128, 'num_csp_blocks': 1, 'use_depthwise': False, 'upsample_cfg': {'scale_factor': 2, 'mode': 'nearest'}, 'norm_cfg': {'type': 'BN', 'momentum': 0.03, 'eps': 0.001}, 'act_cfg': {'type': 'Swish'}}, 'bbox_head': {'type': 'YOLOXHead', 'num_classes': 1, 'in_channels': 128, 'feat_channels': 128, 'stacked_convs': 2, 'strides': (8, 16, 32), 'use_depthwise': False, 'norm_cfg': {'type': 'BN', 'momentum': 0.03, 'eps': 0.001}, 'act_cfg': {'type': 'Swish'}, 'loss_cls

In [869]:
# data_root = 'data/coco_poisoned_0_mixcolored/train2017'
data_root = 'data/coco_poisoned_20_mixcolored/val_clean2017'
data_root = 'data/coco_poisoned_20_mixcolored/val_poisoned2017'

img_name = choice(os.listdir(f"{data_root}")) # "000000001655.jpg"
img_name = f"{data_root}/{img_name}"

img = cv2.imread(img_name)

test_pipeline = get_test_pipeline_cfg(model.cfg)
test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
test_pipeline = Compose(test_pipeline)
data_ = dict(img=img, img_id=0)
data_ = test_pipeline(data_)
data_['inputs'] = [data_['inputs']]
data_['data_samples'] = [data_['data_samples']]
data_ = model.data_preprocessor(data_, False)
results = model(**data_, mode='tensor')


31
